In [ ]:
%load_ext autoreload
%load_ext tensorboard
%matplotlib inline

# Measuring Effects of Data shifts in CBMs

## Setup

In [ ]:
import matplotlib
import concepts_xai
import numpy as np
import os
import random
import tensorflow as tf
import yaml
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import rc
from matplotlib import cm
import seaborn as sns
from importlib import reload
from pathlib import Path
import sklearn
import scipy
from sklearn.model_selection import train_test_split

In [ ]:
################################################################################
## Set seeds up for reproducibility
################################################################################

os.environ['PYTHONHASHSEED'] = str(87)
tf.random.set_seed(87)
np.random.seed(87)
random.seed(87)


In [ ]:
################################################################################
## Global Variables Defining Experiment Flow
################################################################################

_LATEX_SYMBOL = "$"  # Change to "$" if working out of server
USE_DSPRITES = True
if USE_DSPRITES:
    RESULTS_DIR = "robustness_results/dsprites"
    DATASETS_DIR = os.path.join("results/dsprites", "datasets/")
else:
    RESULTS_DIR = "robustness_results/shapes3d"
    DATASETS_DIR = os.path.join("results/shapes3d", "datasets/")
    
Path(DATASETS_DIR).mkdir(parents=True, exist_ok=True)
rc('text', usetex=(_LATEX_SYMBOL == "$"))
plt.style.use('seaborn-whitegrid')


## Utility Functions

In [ ]:
def serialize_results(results_dict, results_dir):
    for result_name, result_arr in results_dict.items():
        np.savez(
            os.path.join(results_dir, f"{result_name}_means.npz"),
            *list(map(
                lambda x: x[0] if isinstance(x[0], np.ndarray) else np.array(x[0]),
                result_arr
            )),
        )
        np.savez(
            os.path.join(results_dir, f"{result_name}_stds.npz"),
            *list(map(
                lambda x: x[1] if isinstance(x[1], np.ndarray) else np.array(x[1]),
                result_arr
            )),
        )

def serialize_experiment_config(config, results_dir):
    with open(
        os.path.join(results_dir, "config.yaml"),
        'w',
    ) as f:
        f.write(yaml.dump(config, sort_keys=True))

def deserialize_experiment_config(results_dir):
    with open(os.path.join(results_dir, "config.yaml"), 'r') as file:
        return yaml.safe_load(file)

    
def heatmap(data, row_labels, col_labels, ax=None,
            cbar_kw={}, cbarlabel="", **kwargs):
    """
    Create a heatmap from a numpy array and two lists of labels.
    
    Modified from https://matplotlib.org/stable/gallery/images_contours_and_fields/image_annotated_heatmap.html
    
    Parameters
    ----------
    data
        A 2D numpy array of shape (N, M).
    row_labels
        A list or array of length N with the labels for the rows.
    col_labels
        A list or array of length M with the labels for the columns.
    ax
        A `matplotlib.axes.Axes` instance to which the heatmap is plotted.  If
        not provided, use current axes or create a new one.  Optional.
    cbar_kw
        A dictionary with arguments to `matplotlib.Figure.colorbar`.  Optional.
    cbarlabel
        The label for the colorbar.  Optional.
    **kwargs
        All other arguments are forwarded to `imshow`.
    """

    if not ax:
        ax = plt.gca()

    # Plot the heatmap
    im = ax.imshow(data, **kwargs)

    # Create colorbar
    cbar = ax.figure.colorbar(im, ax=ax, **cbar_kw)
    cbar.ax.tick_params(labelsize=15)
    cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom", fontsize=25)

    # We want to show all ticks...
    ax.set_xticks(np.arange(data.shape[1]))
    ax.set_yticks(np.arange(data.shape[0]))
    # ... and label them with the respective list entries.
    ax.set_xticklabels(col_labels, fontsize=25)
    ax.set_yticklabels(row_labels, fontsize=25)

    # Let the horizontal axes labeling appear on top.
    ax.tick_params(
        top=True,
        bottom=False,
        labeltop=True,
        labelbottom=False,
    )

    # Rotate the tick labels and set their alignment.
    plt.setp(
        ax.get_xticklabels(),
        rotation=-30,
        ha="right",
        rotation_mode="anchor",
    )

    # Turn spines off and create white grid.
    ax.grid(False)

    ax.set_xticks(np.arange(data.shape[1]+1)-.5, minor=True)
    ax.set_yticks(np.arange(data.shape[0]+1)-.5, minor=True)
    ax.grid(which="minor", color="w", linestyle='-', linewidth=3)
    ax.tick_params(which="minor", bottom=False, left=False)

    return im, cbar


def annotate_heatmap(im, data=None, valfmt="{x:.2f}",
                     textcolors=("black", "white"),
                     threshold=None, **textkw):
    """
    A function to annotate a heatmap.
    
    Modified from https://matplotlib.org/stable/gallery/images_contours_and_fields/image_annotated_heatmap.html
    
    Parameters
    ----------
    im
        The AxesImage to be labeled.
    data
        Data used to annotate.  If None, the image's data is used.  Optional.
    valfmt
        The format of the annotations inside the heatmap.  This should either
        use the string format method, e.g. "$ {x:.2f}", or be a
        `matplotlib.ticker.Formatter`.  Optional.
    textcolors
        A pair of colors.  The first is used for values below a threshold,
        the second for those above.  Optional.
    threshold
        Value in data units according to which the colors from textcolors are
        applied.  If None (the default) uses the middle of the colormap as
        separation.  Optional.
    **kwargs
        All other arguments are forwarded to each call to `text` used to create
        the text labels.
    """

    if not isinstance(data, (list, np.ndarray)):
        data = im.get_array()

    # Normalize the threshold to the images color range.
    if threshold is not None:
        threshold = im.norm(threshold)
    else:
        threshold = im.norm(data.max())/2.

    # Set default alignment to center, but allow it to be
    # overwritten by textkw.
    kw = dict(horizontalalignment="center",
              verticalalignment="center")
    kw.update(textkw)

    # Get the formatter in case a string is supplied
    if isinstance(valfmt, str):
        valfmt = matplotlib.ticker.StrMethodFormatter(valfmt)

    # Loop over the data and create a `Text` for each "pixel".
    # Change the text's color depending on the data.
    texts = []
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            kw.update(color=textcolors[int(im.norm(data[i, j]) <= threshold)])
            text = im.axes.text(j, i, valfmt(data[i, j], None), fontsize=15, **kw)
            texts.append(text)

    return texts

def bold_text(x):
    if _LATEX_SYMBOL == "$":
        return r"$\textbf{" + x + "}$"
    return x

def normalize_purity(purity, n_concepts):
    return purity / (n_concepts / 2)

## Dependency Tasks Dataset Construction

In [ ]:
if not USE_DSPRITES:
    import tensorflow_datasets as tfds

    shapes3d_train_ds = tfds.load('shapes3d', split='train', shuffle_files=True)
else:
    import concepts_xai.datasets.dSprites as dsprites
    import concepts_xai.datasets.latentFactorData as latentFactorData

In [ ]:

def relabel_categorical(x):
    if len(x.shape) == 1:
        _, unique_inds = np.unique(
            x,
            return_inverse=True,
        )
        return unique_inds
    
    result = x[:, :]
    for i in range(x.shape[-1]):
        _, unique_inds = np.unique(
            x[:, i],
            return_inverse=True,
        )
        result[:, i] = unique_inds
    return result

def cardinality_encoding(card_group_1, card_group_2):
    result_to_encoding = {}
    for i in card_group_1:
        for j in card_group_2:
            result_to_encoding[(i, j)] = len(result_to_encoding)
    return result_to_encoding

def extract_data(
    filter_fn,
    sample_map_fn=lambda x: x,
    concept_map_fn=lambda x: x,
    step=1,
    label_fn=lambda ex: ex['label_shape'] * 8 + ex['label_scale'],
    dataset_path=None,
    force_rerun=False,
):
    if (not force_rerun) and dataset_path and os.path.exists(dataset_path):
        # Them time to load up this dataset!
        ds = np.load(dataset_path)
        return ds["X"], ds["y"], ds["c"]
    num_entries = len(shapes3d_train_ds)
    x_train = []
    y_train = []
    c_train = []
    for i, ex in enumerate(tfds.as_numpy(
        shapes3d_train_ds.shuffle(buffer_size=15000)
    )):
        if i % step != 0:
            continue
        concepts = [
            ex['label_floor_hue'],
            ex['label_wall_hue'],
            ex['label_object_hue'],
            ex['label_scale'],
            ex['label_shape'],
            ex['label_orientation'],
        ]
        if not filter_fn(concepts):
            continue
        print(i, end="\r")

        x_train.append(sample_map_fn(ex['image']))
        y_train.append(label_fn(ex))
        c_train.append(concept_map_fn(concepts))
    x_train = np.stack(x_train, axis=0) / 255.0
    y_train = relabel_categorical(np.stack(y_train, axis=0))
    c_train = relabel_categorical(np.stack(c_train, axis=0))
    if dataset_path:
        # Then serialize it to speed up things next time
        np.savez(
            dataset_path,
            X=x_train,
            y=y_train,
            c=c_train,
        )
    return x_train, y_train, c_train


def generate_dsprites_dataset(
    label_fn,
    filter_fn=None,
    dataset_path=None,
    concept_map_fn=lambda x: x,
    sample_map_fn=lambda x: x,
    dsprites_path="dsprites/dsprites_ndarray_co1sh3sc6or40x32y32_64x64.npz",
    force_reload=False,
):
    if (not force_reload) and dataset_path and os.path.exists(dataset_path):
        # Them time to load up this dataset!
        ds = np.load(dataset_path)
        return (
            (ds["x_train"], ds["y_train"], ds["c_train"]),
            (ds["x_test"], ds["y_test"], ds["c_test"])
        )
    
    def _task_fn(x_data, c_data):
        return latentFactorData.get_task_data(
            x_data=x_data,
            c_data=c_data,
            label_fn=label_fn,
            filter_fn=filter_fn,
        )

    loaded_dataset = dsprites.dSprites(
        dataset_path=dsprites_path,
        train_size=0.8,
        random_state=42,
        task=_task_fn,
    )
    _, _, _ = loaded_dataset.load_data()

    x_train = sample_map_fn(loaded_dataset.x_train)
    y_train = loaded_dataset.y_train
    c_train = concept_map_fn(loaded_dataset.c_train)
    
    x_test = sample_map_fn(loaded_dataset.x_test)
    y_test = loaded_dataset.y_test
    c_test = concept_map_fn(loaded_dataset.c_test)
    
    if dataset_path:
        # Then serialize it to speed up things next time
        np.savez(
            dataset_path,
            x_train=x_train,
            y_train=y_train,
            c_train=c_train,
            x_test=x_test,
            y_test=y_test,
            c_test=c_test,
        )
    return (x_train, y_train, c_train), (x_test, y_test, c_test),

## Dataset Construction

In [ ]:
if not USE_DSPRITES:
    ############################################################################
    ## Construct a binary concept task in the shapes3D dataset 
    ############################################################################
    def count_class_balance(y):
        one_hot = tf.keras.utils.to_categorical(y)
        return np.sum(one_hot, axis=0) / one_hot.shape[0]

    def multiclass_task_bin_concepts_map_fn(concepts):
        return [
            int(concepts[0] < 5),
            int(concepts[1] < 5),
            int(concepts[2] < 5),
            int(concepts[3] < 4),
            int(concepts[4] < 2),
            int(concepts[5] < 7),
        ]


    def multiclass_task_bin_concepts_label_fn(concept_dict):
        concept_vector = multiclass_task_bin_concepts_map_fn([
            concept_dict['label_floor_hue'],
            concept_dict['label_wall_hue'],
            concept_dict['label_object_hue'],
            concept_dict['label_scale'],
            concept_dict['label_shape'],
            concept_dict['label_orientation'],
        ])
        binary_label_encoding = [
            concept_vector[0] or concept_vector[1],
            concept_vector[2] or concept_vector[3],
            concept_vector[4] or concept_vector[5],
        ]
        return int(
            "".join(list(map(str, binary_label_encoding))),
            2
        )

    def filter_fn_dep_0(concept):
        ranges = [
            list(range(0, 10, 2)),
            list(range(0, 10, 2)),
            list(range(0, 10, 2)),
            list(range(0, 8)),
            list(range(4)),
            list(range(0, 15, 4)),
        ]

        concept_vector = multiclass_task_bin_concepts_map_fn(concept)
        # First filter as in small dataset to constraint the size of the data a bit
        return all([
            (concept[i] in ranges[i]) for i in range(len(ranges))
        ])



    floor_hue_wall_hue_sets_lower = [
        list(np.random.permutation(7))[:5]
        for _ in range(10)
    ]

    floor_hue_wall_hue_sets_upper = [
        list(3 + np.random.permutation(7))[:5]
        for _ in range(10)
    ]

    def filter_fn_dep_1(concept):
        ranges = [
            list(range(0, 10)),
            list(range(0, 10, 2)),
            list(range(0, 10, 2)),
            list(range(0, 8)),
            list(range(4)),
            list(range(0, 15, 4)),
        ]

        concept_vector = multiclass_task_bin_concepts_map_fn(concept)

        # First filter as in small dataset to constraint the size of the data a bit
        if not all([
            (concept[i] in ranges[i]) for i in range(len(ranges))
        ]):
            return False
        if concept[0] < 5:
            return concept[1] in floor_hue_wall_hue_sets_lower[concept[0]]
        else:
            return (concept[1] in floor_hue_wall_hue_sets_upper[concept[0]])


        wall_hue_object_hue_sets_lower = [
        list(np.random.permutation(7))[:5]
        for _ in range(10)
    ]
    wall_hue_object_hue_sets_upper = [
        list(3 + np.random.permutation(7))[:5]
        for _ in range(10)
    ]

    def filter_fn_dep_2(concept):
        ranges = [
            list(range(0, 10)),
            list(range(0, 10)),
            list(range(0, 10, 2)),
            list(range(0, 8)),
            list(range(4)),
            list(range(0, 15, 4)),
        ]

        concept_vector = multiclass_task_bin_concepts_map_fn(concept)
        # First filter as in small dataset to constraint the size of the data a bit
        if not all([
            (concept[i] in ranges[i]) for i in range(len(ranges))
        ]):
            return False

        if (concept[0] < 5):
            if concept[1] not in floor_hue_wall_hue_sets_lower[concept[0]]:
                return False
        else:
            if (concept[1] not in floor_hue_wall_hue_sets_upper[concept[0]]):
                return False

        if (concept[1] < 5):
            if concept[2] not in wall_hue_object_hue_sets_lower[concept[1]]:
                return False
        else:
            if (concept[2] not in wall_hue_object_hue_sets_upper[concept[1]]):
                return False
        return True


        object_hue_scale_sets_lower = [
        list(np.random.permutation(6))[:4]
        for _ in range(10)
    ]

    object_hue_scale_sets_upper = [
        list(2 + np.random.permutation(6))[:4]
        for _ in range(10)
    ]

    def filter_fn_dep_3(concept):
        ranges = [
            list(range(0, 10)),
            list(range(0, 10)),
            list(range(0, 10)),
            list(range(0, 8)),
            list(range(4)),
            list(range(0, 15, 4)),
        ]

        concept_vector = multiclass_task_bin_concepts_map_fn(concept)

        # First filter as in small dataset to constraint the size of the data a bit
        if not all([
            (concept[i] in ranges[i]) for i in range(len(ranges))
        ]):
            return False

        if (concept[0] < 5):
            if concept[1] not in floor_hue_wall_hue_sets_lower[concept[0]]:
                return False
        else:
            if (concept[1] not in floor_hue_wall_hue_sets_upper[concept[0]]):
                return False

        if (concept[1] < 5):
            if concept[2] not in wall_hue_object_hue_sets_lower[concept[1]]:
                return False
        else:
            if (concept[2] not in wall_hue_object_hue_sets_upper[concept[1]]):
                return False

        if (concept[2] < 5):
            if concept[3] not in object_hue_scale_sets_lower[concept[2]]:
                return False
        else:
            if (concept[3] not in object_hue_scale_sets_upper[concept[2]]):
                return False

        return True


    scale_shape_sets_lower = [
        list(np.random.permutation(3))[:2]
        for _ in range(8)
    ]

    scale_shape_sets_upper = [
        list(1 + np.random.permutation(3))[:2]
        for _ in range(8)
    ]

    def filter_fn_dep_4(concept):
        ranges = [
            list(range(0, 10)),
            list(range(0, 10)),
            list(range(0, 10)),
            list(range(0, 8)),
            list(range(4)),
            list(range(0, 15, 2)),
        ]

        concept_vector = multiclass_task_bin_concepts_map_fn(concept)

        # First filter as in small dataset to constraint the size of the data a bit
        if not all([
            (concept[i] in ranges[i]) for i in range(len(ranges))
        ]):
            return False

        if (concept[0] < 5):
            if concept[1] not in floor_hue_wall_hue_sets_lower[concept[0]]:
                return False
        else:
            if (concept[1] not in floor_hue_wall_hue_sets_upper[concept[0]]):
                return False

        if (concept[1] < 5):
            if concept[2] not in wall_hue_object_hue_sets_lower[concept[1]]:
                return False
        else:
            if (concept[2] not in wall_hue_object_hue_sets_upper[concept[1]]):
                return False

        if (concept[2] < 5):
            if concept[3] not in object_hue_scale_sets_lower[concept[2]]:
                return False
        else:
            if (concept[3] not in object_hue_scale_sets_upper[concept[2]]):
                return False

        if (concept[3] < 4):
            if concept[4] not in scale_shape_sets_lower[concept[3]]:
                return False
        else:
            if (concept[4] not in scale_shape_sets_upper[concept[3]]):
                return False
        return True



    shape_rotation_sets_lower = [
        list(np.random.permutation(9))[:7]
        for _ in range(4)
    ]

    shape_rotation_sets_upper = [
        list(6 + np.random.permutation(9))[:7]
        for _ in range(4)
    ]

    def filter_fn_dep_5(concept):
        ranges = [
            list(range(0, 10)),
            list(range(0, 10)),
            list(range(0, 10)),
            list(range(0, 8)),
            list(range(4)),
            list(range(0, 15)),
        ]

        concept_vector = multiclass_task_bin_concepts_map_fn(concept)

        # First filter as in small dataset to constraint the size of the data a bit
        if not all([
            (concept[i] in ranges[i]) for i in range(len(ranges))
        ]):
            return False

        if (concept[0] < 5):
            if concept[1] not in floor_hue_wall_hue_sets_lower[concept[0]]:
                return False
        else:
            if (concept[1] not in floor_hue_wall_hue_sets_upper[concept[0]]):
                return False

        if (concept[1] < 5):
            if concept[2] not in wall_hue_object_hue_sets_lower[concept[1]]:
                return False
        else:
            if (concept[2] not in wall_hue_object_hue_sets_upper[concept[1]]):
                return False

        if (concept[2] < 5):
            if concept[3] not in object_hue_scale_sets_lower[concept[2]]:
                return False
        else:
            if (concept[3] not in object_hue_scale_sets_upper[concept[2]]):
                return False

        if (concept[3] < 4):
            if concept[4] not in scale_shape_sets_lower[concept[3]]:
                return False
        else:
            if (concept[4] not in scale_shape_sets_upper[concept[3]]):
                return False

        if (concept[4] < 2):
            if concept[5] not in shape_rotation_sets_lower[concept[4]]:
                return False
        else:
            if (concept[5] not in shape_rotation_sets_upper[concept[4]]):
                return False
        return True
else:
    def count_class_balance(y):
        one_hot = tf.keras.utils.to_categorical(y)
        return np.sum(one_hot, axis=0) / one_hot.shape[0]

    def multiclass_binary_concepts_map_fn(concepts):
        new_concepts = np.zeros((concepts.shape[0], 5))
        # We will have 5 concepts:
        # (0) "is it ellipse or square?"
        new_concepts[:, 0] = (concepts[:, 0] < 2).astype(np.int)

        # (1) "is_size < 3?"
        num_sizes = len(set(concepts[:, 1]))
        new_concepts[:, 1] = (concepts[:, 1] < num_sizes/2).astype(np.int)

        # (2) "is rotation < PI/2?"
        num_rots = len(set(concepts[:, 2]))
        new_concepts[:, 2] = (concepts[:, 2] < num_rots/2).astype(np.int)

        # (3) "is x <= 16?"
        num_x_coords = len(set(concepts[:, 3]))
        new_concepts[:, 3] = (concepts[:, 3] < num_x_coords // 2).astype(np.int)

        # (4) "is y <= 16?"
        num_y_coords = len(set(concepts[:, 4]))
        new_concepts[:, 4] = (concepts[:, 4] < num_y_coords // 2).astype(np.int)

        return new_concepts

    def _get_concept_vector(c_data):
        return np.array([
            # First check if it is an ellipse or a square
            int(c_data[0] < 2),
            # Now check that it is "small"
            int(c_data[1] < 3),
            # And it has not been rotated more than PI/2 radians
            int(c_data[2] < 20),
            # Finally, check whether it is in not in the the upper-left quadrant
            int(c_data[3] < 15),
            int(c_data[4] < 15),
        ])

    def multiclass_task_label_fn(c_data):
        # Our task will be a binary task where we are interested in determining
        # whether an image is a "small" ellipse not in the upper-left
        # quadrant that has been rotated less than 3*PI/2 radians
        concept_vector = _get_concept_vector(c_data)
        binary_label_encoding = [
            concept_vector[0] or concept_vector[1],
            concept_vector[2] or concept_vector[3],
            concept_vector[4],
        ]
        return int(
            "".join(list(map(str, binary_label_encoding))),
            2
        )

    def dep_0_filter_fn(concept):
        ranges = [
            list(range(3)),
            list(range(0, 6, 2)),
            list(range(0, 40, 4)),
            list(range(0, 32, 2)),
            list(range(0, 32, 2)),
        ]
        return all([
            (concept[i] in ranges[i]) for i in range(len(ranges))
        ])



    scale_shape_sets_lower = [
        list(np.random.permutation(4))[:3] for i in range(3)
    ]

    scale_shape_sets_upper = [
        list(2 + np.random.permutation(4))[:3] for i in range(3)
    ]
    def dep_1_filter_fn(concept):
        ranges = [
            list(range(3)),
            list(range(6)),
            list(range(0, 40, 4)),
            list(range(0, 32, 2)),
            list(range(0, 32, 2)),
        ]

        concept_vector = _get_concept_vector(concept)

        # First filter as in small dataset to constraint the size of the data a bit
        if not all([
            (concept[i] in ranges[i]) for i in range(len(ranges))
        ]):
            return False
        if concept_vector[0]:
            if concept[1] not in scale_shape_sets_lower[concept[0]]:
                return False
        else:
            if concept[0] not in scale_shape_sets_upper[concept[0]]:
                return False
        return True



    rotation_scale_sets_lower = [
        list(np.random.permutation(30))[:20] for i in range(6)
    ]

    rotation_scale_sets_upper = [
        list(10 + np.random.permutation(30))[:20] for i in range(6)
    ]
    def dep_2_filter_fn(concept):
        ranges = [
            list(range(3)),
            list(range(6)),
            list(range(0, 40, 2)),
            list(range(0, 32, 2)),
            list(range(0, 32, 2)),
        ]

        concept_vector = _get_concept_vector(concept)

        # First filter as in small dataset to constraint the size of the data a bit
        if not all([
            (concept[i] in ranges[i]) for i in range(len(ranges))
        ]):
            return False
        if concept_vector[0]:
            if concept[1] not in scale_shape_sets_lower[concept[0]]:
                return False
        else:
            if concept[0] not in scale_shape_sets_upper[concept[0]]:
                return False

        if concept_vector[1]:
            if concept[2] not in rotation_scale_sets_lower[concept[1]]:
                return False
        else:
            if concept[2] not in rotation_scale_sets_upper[concept[1]]:
                return False
        return True



    x_pos_rotation_sets_lower = [
        list(np.random.permutation(20))[:16]
        for i in range(40)
    ]

    x_pos_rotation_sets_upper = [
        list(12 + np.random.permutation(20))[:16]
        for i in range(40)
    ]
    def dep_3_filter_fn(concept):
        ranges = [
            list(range(3)),
            list(range(6)),
            list(range(0, 40, 2)),
            list(range(0, 32)),
            list(range(0, 32, 2)),
        ]

        concept_vector = _get_concept_vector(concept)

        # First filter as in small dataset to constraint the size of the data a bit
        if not all([
            (concept[i] in ranges[i]) for i in range(len(ranges))
        ]):
            return False
        if concept_vector[0]:
            if concept[1] not in scale_shape_sets_lower[concept[0]]:
                return False
        else:
            if concept[0] not in scale_shape_sets_upper[concept[0]]:
                return False

        if concept_vector[1]:
            if concept[2] not in rotation_scale_sets_lower[concept[1]]:
                return False
        else:
            if concept[2] not in rotation_scale_sets_upper[concept[1]]:
                return False

        if concept_vector[2]:
            if concept[3] not in x_pos_rotation_sets_lower[concept[2]]:
                return False
        else:
            if concept[3] not in x_pos_rotation_sets_upper[concept[2]]:
                return False
        return True

    y_pos_x_pos_sets_lower = [
        list(np.random.permutation(20))[:16]
        for i in range(32)
    ]

    y_pos_x_pos_sets_upper = [
        list(12 + np.random.permutation(20))[:16]
        for i in range(32)
    ]
    def dep_4_filter_fn(concept):
        ranges = [
            list(range(3)),
            list(range(6)),
            list(range(0, 40, 2)),
            list(range(0, 32)),
            list(range(0, 32)),
        ]

        concept_vector = _get_concept_vector(concept)

        # First filter as in small dataset to constraint the size of the data a bit
        if not all([
            (concept[i] in ranges[i]) for i in range(len(ranges))
        ]):
            return False
        if concept_vector[0]:
            if concept[1] not in scale_shape_sets_lower[concept[0]]:
                return False
        else:
            if concept[0] not in scale_shape_sets_upper[concept[0]]:
                return False

        if concept_vector[1]:
            if concept[2] not in rotation_scale_sets_lower[concept[1]]:
                return False
        else:
            if concept[2] not in rotation_scale_sets_upper[concept[1]]:
                return False

        if concept_vector[2]:
            if concept[3] not in x_pos_rotation_sets_lower[concept[2]]:
                return False
        else:
            if concept[3] not in x_pos_rotation_sets_upper[concept[2]]:
                return False

        if concept_vector[3]:
            if concept[4] not in y_pos_x_pos_sets_lower[concept[3]]:
                return False
        else:
            if concept[4] not in y_pos_x_pos_sets_upper[concept[3]]:
                return False
        return True

In [ ]:
_FORCE_RERUN = False
if not USE_DSPRITES:
    _FORCE_RERUN = False
if not USE_DSPRITES:
    def balanced_multiclass_task_bin_concepts_label_fn(concept_dict):
        concept_vector = multiclass_task_bin_concepts_map_fn([
            concept_dict['label_floor_hue'],
            concept_dict['label_wall_hue'],
            concept_dict['label_object_hue'],
            concept_dict['label_scale'],
            concept_dict['label_shape'],
            concept_dict['label_orientation'],
        ])
        if concept_vector[4] == 0:
            offset = 0
            binary_label_encoding = [
                concept_vector[0],
                concept_vector[1],
            ]
        else:
            offset = 4
            binary_label_encoding = [
                concept_vector[2],
                concept_vector[3],
                concept_vector[5],
            ]

        return offset + int(
            "".join(list(map(str, binary_label_encoding))),
            2
        )

    data_x_train, data_y_train, data_c_train = extract_data(
        filter_fn_dep_0,
        dataset_path=os.path.join(DATASETS_DIR, "balanced_multiclass_task_bin_concepts_dep_0_concepts.npz"),
        concept_map_fn=multiclass_task_bin_concepts_map_fn,
        label_fn=balanced_multiclass_task_bin_concepts_label_fn,
        force_rerun=_FORCE_RERUN,
    )

    data_x_train, data_x_test, data_y_train, data_y_test, data_c_train, data_c_test = train_test_split(
        data_x_train,
        data_y_train,
        data_c_train,
        test_size=0.2,
    )

    print("data_x_train.shape =", data_x_train.shape)
    print("data_x_train label distribution:", count_class_balance(data_y_train))
    print("data_x_train concept distribution:", np.sum(data_c_train, axis=0)/data_c_train.shape[0])
    print("data_x_train exclusive concept distribution:", np.sum(data_c_train[np.sum(data_c_train, axis=-1) == 1], axis=0)/data_c_train.shape[0])
    print("data_x_test.shape =", data_x_test.shape)
    print("data_c_test.shape =", data_c_test.shape)
    data_train = (data_x_train, data_y_train, data_c_train)
    data_test = (data_x_test, data_y_test, data_c_test)

    dep_0_corr_mat = np.ones(
        (
            data_train[2].shape[-1],
            len(set(data_train[1]))
        )
    )
    for c in range(dep_0_corr_mat.shape[0]):
        for l in range(dep_0_corr_mat.shape[1]):
            dep_0_corr_mat[c][l] = np.corrcoef(
                data_train[2][:, c],
                (data_train[1] == l).astype(np.int32),
            )[0, 1]
    fig, ax = plt.subplots(1, figsize=(8, 6))
    im, cbar = heatmap(
        np.abs(dep_0_corr_mat),
        [f"$c_{i}$" for i in range(dep_0_corr_mat.shape[0])],
        [f"$l_{i}$" for i in range(dep_0_corr_mat.shape[1])],
        ax=ax,
        cmap="magma",
        cbarlabel=f"Correlation Coef",
        vmin=0, #-1,
        vmax=1,
    )
    texts = annotate_heatmap(im, valfmt="{x:.2f}")
    fig.tight_layout()

    fig.suptitle(f"3dshapes Concept-Label Absolute Correlations ($\lambda = 0$)", fontsize=25)
    fig.subplots_adjust(top=0.85)
    plt.show()

    print("")
else:
    def balanced_multiclass_task_label_fn(c_data):
        # Our task will be a binary task where we are interested in determining
        # whether an image is a "small" ellipse not in the upper-left
        # quadrant that has been rotated less than 3*PI/2 radians
        concept_vector = _get_concept_vector(c_data)
        threshold = 0
        if concept_vector[0] == 1:
            binary_label_encoding = [
                concept_vector[1],
                concept_vector[2],
            ]
        else:
            threshold = 4
            binary_label_encoding = [
                concept_vector[3],
                concept_vector[4],
            ]
        return threshold + int(
            "".join(list(map(str, binary_label_encoding))),
            2
        )
    
    def robust_label_example(c_data):
        concept_vector = _get_concept_vector(c_data)
        return np.sum(concept_vector)

    data_train, data_test = generate_dsprites_dataset(
        label_fn=balanced_multiclass_task_label_fn,
        filter_fn=dep_0_filter_fn,
        dataset_path=os.path.join(DATASETS_DIR, "balanced_multiclass_task_bin_concepts_dep_0_complete_dataset.npz"),
        dsprites_path="dsprites/dsprites_ndarray_co1sh3sc6or40x32y32_64x64.npz",
        concept_map_fn=multiclass_binary_concepts_map_fn,
    #     force_reload=True,
    )

    dep_0_corr_mat = np.ones(
        (
            data_train[2].shape[-1],
            len(set(data_train[1]))
        )
    )
    for c in range(dep_0_corr_mat.shape[0]):
        for l in range(dep_0_corr_mat.shape[1]):
            dep_0_corr_mat[c][l] = np.corrcoef(
                data_train[2][:, c],
                (data_train[1] == l).astype(np.int32),
            )[0, 1]
    fig, ax = plt.subplots(1, figsize=(8, 6))
    im, cbar = heatmap(
        np.abs(dep_0_corr_mat),
        [f"$c_{i}$" for i in range(dep_0_corr_mat.shape[0])],
        [f"$l_{i}$" for i in range(dep_0_corr_mat.shape[1])],
        ax=ax,
        cmap="magma",
        cbarlabel=f"Correlation Coef",
        vmin=0,
        vmax=1,
    )
    texts = annotate_heatmap(im, valfmt="{x:.2f}")
    fig.tight_layout()

    fig.suptitle(f"dSprites Concept-Label Absolute Correlations ($\lambda = 0$)", fontsize=25)
    fig.subplots_adjust(top=0.85)
    plt.show()

    print("data_dataset train size:", data_train[0].shape[0])
    print("data_dataset train concept size:", data_train[2].shape)
    print("\tTrain balance:", count_class_balance(data_train[1]))
    print("\tConcept balance:", np.sum(data_train[2], axis=0)/data_train[2].shape[0])
    print("")
    
    
    robust_data_train, robust_data_test = generate_dsprites_dataset(
        label_fn=robust_label_example,
        filter_fn=dep_0_filter_fn,
        dataset_path=os.path.join(DATASETS_DIR, "robust_task_bin_concepts_dep_0_dataset.npz"),
        dsprites_path="dsprites/dsprites_ndarray_co1sh3sc6or40x32y32_64x64.npz",
        concept_map_fn=multiclass_binary_concepts_map_fn,
    #     force_reload=True,
    )

    robust_corr_mat = np.ones(
        (
            robust_data_train[2].shape[-1],
            len(set(robust_data_train[1]))
        )
    )
    for c in range(robust_corr_mat.shape[0]):
        for l in range(robust_corr_mat.shape[1]):
            robust_corr_mat[c][l] = np.corrcoef(
                robust_data_train[2][:, c],
                (robust_data_train[1] == l).astype(np.int32),
            )[0, 1]
    fig, ax = plt.subplots(1, figsize=(8, 6))
    im, cbar = heatmap(
        np.abs(robust_corr_mat),
        [f"$c_{i}$" for i in range(robust_corr_mat.shape[0])],
        [f"$l_{i}$" for i in range(robust_corr_mat.shape[1])],
        ax=ax,
        cmap="magma",
        cbarlabel=f"Correlation Coef",
        vmin=0,
        vmax=1,
    )
    texts = annotate_heatmap(im, valfmt="{x:.2f}")
    fig.tight_layout()

    fig.suptitle(f"dSprites Robust Dataset ($\lambda = 0$)", fontsize=25)
    fig.subplots_adjust(top=0.85)
    plt.show()

    print("robust_data_dataset train size:", robust_data_train[0].shape[0])
    print("robust_data_dataset train concept size:", robust_data_train[2].shape)
    print("\tTrain balance:", count_class_balance(robust_data_train[1]))
    print("\tConcept balance:", np.sum(robust_data_train[2], axis=0)/robust_data_train[2].shape[0])
    print("")


## Model Construction

In [ ]:
# Construct the encoder model
def _extract_concepts(activations, concept_cardinality):
    concepts = []
    total_seen = 0
    if all(np.array(concept_cardinality) <= 2):
        # Then nothing to do here as they are all binary concepts
        return activations
    for num_values in concept_cardinality:
        concepts.append(activations[:, total_seen: total_seen + num_values])
        total_seen += num_values
    return concepts
    
def construct_encoder(
    input_shape,
    filter_groups,
    units,
    concept_cardinality,
    drop_prob=0.5,
    max_pool_window=(2,2),
    max_pool_stride=2,
    latent_dims=0,
    output_logits=False,
):
    encoder_inputs = tf.keras.Input(shape=input_shape)
    encoder_compute_graph = encoder_inputs
    
    # Start with our convolutions
    num_convs = 0
    for filter_group in filter_groups:
        for (num_filters, kernel_size) in filter_group:
            encoder_compute_graph = tf.keras.layers.Conv2D(
                filters=num_filters,
                kernel_size=kernel_size,
                padding="SAME",
                activation=None,
                name=f'encoder_conv_{num_convs}',
            )(encoder_compute_graph)
            num_convs += 1
            encoder_compute_graph = tf.keras.layers.BatchNormalization()(
                encoder_compute_graph
            )
            encoder_compute_graph = tf.keras.activations.relu(encoder_compute_graph)
        # Then do a max pool here to control the parameter count of the model
        # at the end of each group
        encoder_compute_graph = tf.keras.layers.MaxPooling2D(
            pool_size=max_pool_window,
            strides=max_pool_stride,
        )(
            encoder_compute_graph
        )
    
    # Flatten this guy
    encoder_compute_graph = tf.keras.layers.Flatten()(encoder_compute_graph)
    
    # Add a dropout if requested
    if drop_prob:
        encoder_compute_graph = tf.keras.layers.Dropout(drop_prob)(
            encoder_compute_graph
        )
    
    # Finally, include the fully connected bottleneck here
    for i, units in enumerate(units):
        encoder_compute_graph = tf.keras.layers.Dense(
            units,
            activation='relu',
            name=f"encoder_dense_{i}",
        )(encoder_compute_graph)
    
    if latent_dims:
        bypass = tf.keras.layers.Dense(
            latent_dims,
            activation="sigmoid",
            name="encoder_bypass_channel",
        )(encoder_compute_graph)
    else:
        bypass = None
    
    # Map to our output distribution to a flattened
    # vector where we will extract distributions over
    # all concept values
    encoder_compute_graph = tf.keras.layers.Dense(
        sum(concept_cardinality),
        activation=None,
        name="encoder_concept_outputs",
    )(encoder_compute_graph)
        
    # Separate this vector into all of its heads
    concept_outputs = _extract_concepts(
        encoder_compute_graph,
        concept_cardinality,
    )
    if not output_logits:
        if isinstance(concept_outputs, list):
            for i, concept_vec in enumerate(concept_outputs):
                if concept_vec.shape[-1] == 1:
                    # Then this is a binary concept so simply apply sigmoid
                    concept_outputs[i] = tf.keras.activations.sigmoid(concept_vec)
                else:
                    # Else we will apply a softmax layer as we assume that all of these
                    # entries represent a multi-modal probability distribution
                    concept_outputs[i] = tf.keras.activations.softmax(
                        concept_vec,
                        axis=-1,
                    )
        else:
            # Else they are allbinary concepts so let's sigmoid them
            concept_outputs = tf.keras.activations.sigmoid(concept_outputs)
    return tf.keras.Model(
        encoder_inputs,
        [concept_outputs, bypass] if bypass is not None else concept_outputs,
        name="encoder",
    )

In [ ]:
############################################################################
## Build concepts-to-labels model
############################################################################

def construct_decoder(units, num_outputs):
    decoder_layers = [tf.keras.layers.Flatten()] + [
        tf.keras.layers.Dense(
            units,
            activation=tf.nn.relu,
            name=f"decoder_dense_{i+1}",
        ) for i, units in enumerate(units)
    ]
    return tf.keras.Sequential(decoder_layers + [
        tf.keras.layers.Dense(
            num_outputs if num_outputs > 2 else 1,
            activation=None,
            name="decoder_model_output",
        )
    ])

In [ ]:
# Construct the complete model
def construct_end_to_end_model(
    input_shape,
    encoder,
    decoder,
    num_outputs,
    learning_rate=1e-3,
):
    model_inputs = tf.keras.Input(shape=input_shape)
    latent = encoder(model_inputs)
    if isinstance(latent, list):
        if len(latent) > 1:
            compacted_vector = tf.keras.layers.Concatenate(axis=-1)(
                latent
            )
        else:
            compacted_vector = latent[0]
    else:
        compacted_vector = latent
    model_compute_graph = decoder(compacted_vector)
    # Now time to collapse all the concepts again back into a single vector
    model = tf.keras.Model(
        model_inputs,
        model_compute_graph,
        name="complete_model",
    )
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate),
        loss=(
            tf.keras.losses.BinaryCrossentropy(from_logits=True) if (num_outputs <= 2)
            else tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
        ),
        metrics=[
            "binary_accuracy" if (num_outputs <= 2)
            else tf.keras.metrics.SparseTopKCategoricalAccuracy(k=1)
        ],
    )
    return model, encoder, decoder

In [ ]:
############################################################################
## Build CBM
############################################################################
import concepts_xai.methods.CBM.CBModel as CBM

def construct_cbm(
    encoder,
    decoder,
    num_outputs,
    alpha=0.1,
    learning_rate=1e-3,
    latent_dims=0,
    encoder_output_logits=False,
):
    model_factory = CBM.BypassJointCBM if latent_dims else CBM.JointConceptBottleneckModel
    cbm_model = model_factory(
        encoder=encoder,
        decoder=decoder,
        task_loss=(
            tf.keras.losses.BinaryCrossentropy(from_logits=True) if (num_outputs <= 2)
            else tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
        ),
        name="joint_cbm",
        metrics=[
            tf.keras.metrics.BinaryAccuracy() if (num_outputs <= 2)
            else tf.keras.metrics.SparseTopKCategoricalAccuracy(k=1)
        ],
        alpha=alpha,
        pass_concept_logits=encoder_output_logits,
    )

    ############################################################################
    ## Compile CBM Model
    ############################################################################

    cbm_model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate),
    )
    return cbm_model


In [ ]:
def corrupt_image_color_random(sample, y, prob=0.75):
    width, height = sample.shape[0:2]
    result = sample.copy()
    if np.random.choice([False, True], p=[1-prob, prob]):
        mask = result == 0
        return result + mask * y * 0.1
    return result

def corrupt_dataset_color_random(x, y):
    new_x = x.copy()
    for i, sample in enumerate(new_x):
        new_x[i, :] = corrupt_image_color_random(sample, y[i])
    return new_x

color_random_corrupted_data_train = corrupt_dataset_color_random(data_train[0], data_train[1]), data_train[1], data_train[2]
color_random_corrupted_data_test = corrupt_dataset_color_random(data_test[0], data_test[1]), data_test[1], data_test[2]

for sample, label, i in zip(color_random_corrupted_data_train[0], color_random_corrupted_data_train[1], range(10)):
    print("Sample", i, "with label", label)
    plt.grid(False)
    plt.imshow(sample, cmap='gray', vmin=0, vmax=1)
    plt.show()


In [ ]:
def gallery(array, ncols=3):
    nindex, height, width, intensity = array.shape
    nrows = nindex//ncols
    assert nindex == nrows*ncols
    # want result.shape = (height*nrows, width*ncols, intensity)
    result = (array.reshape(nrows, ncols, height, width, intensity)
              .swapaxes(1,2)
              .reshape(height*nrows, width*ncols, intensity))
    return result

In [ ]:
num_classes = len(set(color_random_corrupted_data_train[1]))
images = [None for _ in range(num_classes)]
seen_classes = set()
for sample, label in zip(*color_random_corrupted_data_train[:-1]):
    if label in seen_classes:
        continue
    seen_classes.add(label)
    images[label] = sample
    if len(seen_classes) == num_classes:
        break

plt.grid(False)
plt.axis(False)
plt.imshow(gallery(np.array(images), ncols=4))
plt.show()

## Experiment Loop

In [ ]:
import concepts_xai.evaluation.metrics.niching as niching
from sklearn.preprocessing import OneHotEncoder
from sklearn.neural_network import MLPClassifier
import scipy
import joblib
import concepts_xai.evaluation.metrics.purity as purity

def construct_trivial_auc_mat(num_concepts):
    result = np.ones((num_concepts, num_concepts), dtype=np.float32) * 0.5
    return result + np.eye(num_concepts, dtype=np.float32) * 0.5
    
    
def cbm_intervention(
    cbm_model,
    x_train,
    x_test,
    y_test,
    c_test,
    intervention_concepts,
):
    # matrix mapping (concept, label) to the value we should set it
    # to during an intervention
    n_concepts = c_test.shape[-1]
    intervention_values = np.zeros((n_concepts, 2))
    if cbm_model.pass_concept_logits:
        # Then the token value we will use for intervening
        # will be based on the percentile of values in the
        # training data
        c_train_pred = cbm_model.encoder(x_train)
        for i in range(n_concepts):
            low_percentile, high_percentile = np.percentile(c_train_pred[:, i], [5, 95])
            intervention_values[i, 0] = low_percentile
            intervention_values[i, 1] = high_percentile
    else:
        # Else we use hard thresholds of 0 and 1 for everything as we are using
        # real probabilities
        intervention_values[:, 1] = 1.0
    
    # Now time to make our concept predictions
    c_test_pred = cbm_model.encoder(x_test).numpy()
    for int_concept in intervention_concepts:
        c_test_pred[:, int_concept] = (
            intervention_values[int_concept, 0] * (c_test[:, int_concept] == 0) +
            intervention_values[int_concept, 1] * (c_test[:, int_concept] == 1)
        )
    
    # Make the actual task predictions
    # Extend c_test_pred so that it has its complement probability as well
    y_test_preds = cbm_model.predict_from_concepts(c_test_pred).numpy()
    y_test_preds = scipy.special.softmax(y_test_preds, axis=-1)
    
    # Time to evaluate these predictions
    return {
        "accuracy": sklearn.metrics.accuracy_score(
            y_test,
            np.argmax(y_test_preds, axis=-1),
        ),
        "auc": sklearn.metrics.roc_auc_score(
            tf.keras.utils.to_categorical(y_test),
            y_test_preds,
            multi_class='ovo',
        ),
    }
    
    
def cbm_mixed_capacity_intervention_experiment_loop(
    train_data,
    corrupted_train_data,
    test_data,
    corrupted_test_data,
    experiment_config,
    load_from_cache=False,
):
    experiment_variables = dict(
        current_experiment_idx=0,
        
        task_accuracies=[],
        task_aucs=[],
        concept_accuracies=[],
        purity_scores=[],
        non_oracle_purity_scores=[],
        purity_matrices=[],
        oracle_matrices=[],
        
        intervention_task_accuracies=[],
        intervention_task_aucs=[],

        niche_impurity_aucs=[],
        niche_impurity_by_betas=[],
        niche_size_by_betas=[],
        
        concept_niche_impurity_aucs=[],
        concept_niche_impurity_by_betas=[],
        concept_niche_size_by_betas=[],
        
        
        
        corrupted_task_accuracies=[],
        corrupted_task_aucs=[],
        corrupted_concept_accuracies=[],
        corrupted_purity_scores=[],
        corrupted_purity_matrices=[],

        corrupted_intervention_task_accuracies=[],
        corrupted_intervention_task_aucs=[],

        corrupted_niche_impurity_aucs=[],
        corrupted_niche_impurity_by_betas=[],
        corrupted_niche_size_by_betas=[],
        
        corrupted_concept_niche_impurity_aucs=[],
        corrupted_concept_niche_impurity_by_betas=[],
        corrupted_concept_niche_size_by_betas=[],
    )
    experiment_config["data_concepts"] = experiment_config.get(
        "data_concepts",
        experiment_config["num_concepts"],
    )
    
    Path(
        os.path.join(
            experiment_config["results_dir"],
            "models",
        )
    ).mkdir(parents=True, exist_ok=True)
    
    x_train, y_train, c_train = train_data
    corrupted_x_train, corrupted_y_train, corrupted_c_train = corrupted_train_data
    x_test, y_test, c_test = test_data
    corrupted_x_test, corrupted_y_test, corrupted_c_test = corrupted_test_data
    start_ind = 0
    if load_from_cache:
        cached = os.path.exists(
            os.path.join(experiment_config['results_dir'], 'results.joblib')
        )
        if cached:
            print(
                "Found cached directory at",
                os.path.join(experiment_config['results_dir'], 'results.joblib'),
            )
            results = joblib.load(
                os.path.join(experiment_config['results_dir'], 'results.joblib')
            )
            start_ind = results['current_experiment_idx']
            print("\tStart index is", start_ind)
            complete_cache = start_ind == len(experiment_config["alpha_values"])
            for varname in experiment_variables.keys():
                if varname == 'current_experiment_idx':
                    continue
                if (varname not in results):
                    print("\tWe could not find variable", varname, "in the cache")
                    complete_cache = False
                    continue
                if len(results[varname]) != start_ind:
                    complete_cache = False
                    start_ind = min(len(results[varname]), start_ind)
            if complete_cache:
                # Then we are good to go
                if not os.path.exists(
                    os.path.join(experiment_config["results_dir"], "config.yaml")
                ):
                    # then serialize the config as this is a different version run
                    serialize_experiment_config(
                        experiment_config,
                        experiment_config["results_dir"],
                    )
                return results
            else:
                # Else we have a partial initialization so let's use it
                experiment_variables = results
    
    # Let's save our config here either way
    serialize_experiment_config(
        experiment_config,
        experiment_config["results_dir"],
    )
    
    verbosity = experiment_config.get("verbosity", 0)
    for alpha in experiment_config["alpha_values"][start_ind:]:
        print("Training with alpha:", alpha)
        task_accs = []
        concept_accs = []
        aucs = []
        purity_mats = []
        oracle_mats = []
        purities = []
        non_oracle_purities = []
        niche_impurity_aucs = []
        niche_impurity_by_betas = []
        niche_size_by_betas = []
        
        concept_niche_impurity_aucs = []
        concept_niche_impurity_by_betas = []
        concept_niche_size_by_betas = []
        
        intervention_task_accuracies = []
        intervention_task_aucs = []
        
        corrupted_task_accs = []
        corrupted_concept_accs = []
        corrupted_aucs = []
        corrupted_purity_mats = []
        corrupted_purities = []
        
        corrupted_niche_impurity_aucs = []
        corrupted_niche_impurity_by_betas = []
        corrupted_niche_size_by_betas = []
        
        corrupted_concept_niche_impurity_aucs = []
        corrupted_concept_niche_impurity_by_betas = []
        corrupted_concept_niche_size_by_betas = []
        
        corrupted_intervention_task_accuracies = []
        corrupted_intervention_task_aucs = []
        
        for trial in range(experiment_config["trials"]):
            print(f"\tTrial {trial + 1}/{experiment_config['trials']} for alpha", alpha)
            # First proceed to do an end-to-end model in case we want to
            # do some task-specific pretraining
            encoder_units = experiment_config["encoder_units"]
            decoder_units = experiment_config["decoder_units"]
            
            end_to_end_model, encoder, decoder = construct_end_to_end_model(
                input_shape=experiment_config["input_shape"],
                num_outputs=experiment_config["num_outputs"],
                learning_rate=experiment_config["learning_rate"],
                encoder=construct_encoder(
                    input_shape=experiment_config["input_shape"],
                    filter_groups=experiment_config["encoder_filter_groups"],
                    units=encoder_units,
                    concept_cardinality=experiment_config["concept_cardinality"],
                    drop_prob=experiment_config.get("drop_prob", 0.5),
                    max_pool_window=experiment_config.get("max_pool_window", (2, 2)),
                    max_pool_stride=experiment_config.get("max_pool_stride", (2, 2)),
                    latent_dims=experiment_config.get("latent_dims", 0),
                    output_logits=experiment_config.get("encoder_output_logits", False),
                ),
                decoder=construct_decoder(
                    units=decoder_units,
                    num_outputs=experiment_config["num_outputs"],
                ),
            )
            
            if experiment_config.get("pre_train_epochs"):
                print("\tModel pre-training with corrupted samples...")
                end_to_end_model.fit(
                    x=corrupted_x_train,
                    y=corrupted_y_train,
                    epochs=experiment_config["pre_train_epochs"],
                    batch_size=experiment_config["batch_size"],
                    validation_split=experiment_config["holdout_fraction"],
                    verbose=verbosity,
                )
                print("\t\tModel pre-training completed")
            
            # Now time to actually construct and train the CBM
            cbm_model = construct_cbm(
                encoder=encoder,
                decoder=decoder,
                alpha=alpha,
                learning_rate=experiment_config["learning_rate"],
                num_outputs=experiment_config["num_outputs"],
                latent_dims=experiment_config.get("latent_dims", 0),
                encoder_output_logits=experiment_config.get("encoder_output_logits", False),
            )

            early_stopping_monitor = tf.keras.callbacks.EarlyStopping(
                monitor=experiment_config.get(
                    "early_stop_metric",
                    "val_concept_accuracy",
                ),
                min_delta=experiment_config["min_delta"],
                patience=experiment_config["patience"],
                restore_best_weights=True,
                verbose=2,
                mode=experiment_config.get(
                    "early_stop_mode",
                    "max",
                ),
            )
            if experiment_config["warmup_epochs"]:
                print("\tWarmup training with corrupted samples...")
                cbm_model.fit(
                    x=corrupted_x_train,
                    y=(corrupted_y_train, corrupted_c_train),
                    epochs=experiment_config["warmup_epochs"],
                    batch_size=experiment_config["batch_size"],
                    validation_split=experiment_config["holdout_fraction"],
                    verbose=verbosity,
                )
                print("\t\tWarmup training completed")


            print("\tCBM training with corrupted samples...")
            cbm_model.fit(
                x=corrupted_x_train,
                y=(corrupted_y_train, corrupted_c_train),
                epochs=experiment_config["max_epochs"],
                batch_size=experiment_config["batch_size"],
                callbacks=[
                    early_stopping_monitor,
                ],
                validation_split=experiment_config["holdout_fraction"],
                verbose=verbosity,
            )
            print("\t\tCBM training completed")
            print("\tSerializing model")
            encoder.save(
                os.path.join(
                    experiment_config["results_dir"],
                    f"models/encoder_{alpha}_trial_{trial}"
                )
            )
            decoder.save(
                os.path.join(
                    experiment_config["results_dir"],
                    f"models/decoder_{alpha}_trial_{trial}"
                )
            )
            
            print("\tEvaluating model (corrupted data)")
            # First evaluate with corrupted samples
            corrupted_test_result = cbm_model.evaluate(
                corrupted_x_test,
                (
                    corrupted_y_test,
                    corrupted_c_test[:, :experiment_config["num_concepts"]],
                ),
                verbose=0,
                return_dict=True,
            )
            corrupted_task_accs.append(
                corrupted_test_result['sparse_top_k_categorical_accuracy']
                if experiment_config['num_outputs'] > 1 else
                corrupted_test_result['binary_accuracy']
            )
            corrupted_concept_accs.append(corrupted_test_result['concept_accuracy'])
            
            if experiment_config['num_outputs'] > 1:
                # Then lets apply a softmax activation over all the probability
                # classes
                preds = scipy.special.softmax(cbm_model.predict(corrupted_x_test)[0], axis=-1)
                print("preds.shape =", preds.shape)
                # And select just the labels that are in fact being used
                one_hot_labels = tf.keras.utils.to_categorical(corrupted_y_test)
                print("one_hot_labels.shape =", one_hot_labels.shape)
                corrupted_aucs.append(sklearn.metrics.roc_auc_score(
                    one_hot_labels,
                    preds,
                    multi_class='ovo',
                ))
            else:
                corrupted_aucs.append(sklearn.metrics.roc_auc_score(
                    corrupted_y_test,
                    cbm_model.predict(corrupted_x_test)[0],
                ))
            
            print(
                f"\t\tCorrupted Test auc = {corrupted_aucs[-1]:.4f}, "
                f"corrupted test concept accuracy = {corrupted_concept_accs[-1]:.4f}, "
                f"corrupted task accuracy = {corrupted_task_accs[-1]:.4f}"
            )
            
            
            
            
            print("\tEvaluating model (real data)")
            test_result = cbm_model.evaluate(
                x_test,
                (
                    y_test,
                    c_test[:, :experiment_config["num_concepts"]],
                ),
                verbose=0,
                return_dict=True,
            )
            task_accs.append(
                test_result['sparse_top_k_categorical_accuracy']
                if experiment_config['num_outputs'] > 1 else
                test_result['binary_accuracy']
            )
            concept_accs.append(test_result['concept_accuracy'])

            if experiment_config['num_outputs'] > 1:
                # Then lets apply a softmax activation over all the probability
                # classes
                preds = scipy.special.softmax(cbm_model.predict(x_test)[0], axis=-1)
                print("preds.shape =", preds.shape)
                
                # And select just the labels that are in fact being used
                one_hot_labels = tf.keras.utils.to_categorical(y_test)
                print("one_hot_labels.shape =", one_hot_labels.shape)
                aucs.append(sklearn.metrics.roc_auc_score(
                    one_hot_labels,
                    preds,
                    multi_class='ovo',
                ))
            else:
                aucs.append(sklearn.metrics.roc_auc_score(
                    y_test,
                    cbm_model.predict(x_test)[0],
                ))

            print(
                f"\t\tReal Test auc = {aucs[-1]:.4f}, "
                f"real test concept accuracy = {concept_accs[-1]:.4f}, "
                f"real task accuracy = {task_accs[-1]:.4f}"
            )
            
            

            
            print(f"\t\tComputing purity score (real data)...")
            c_train_pred = cbm_model.encoder(x_train).numpy()
            c_test_pred = cbm_model.encoder(x_test).numpy()

            enc = OneHotEncoder(sparse=False)
            y_train_one_hot = enc.fit_transform(y_train.reshape(-1, 1))
            y_test_one_hot = enc.fit_transform(y_test.reshape(-1, 1))

            # END TASK NICHING SCORES
            classifier = MLPClassifier((20, 20), random_state=1, max_iter=1000).fit(c_train_pred, y_train_one_hot)
            current_niche_impurity_beta_scores = []
            current_niche_size_beta_scores = []
            # And estimate the area under the curve using the trapezoid method
            current_niche_impurity_auc = 0
            prev_value_map = {}
            delta_beta = experiment_config.get("delta_beta", 0.05)
            for beta in np.arange(0.0, 1.0, experiment_config.get("delta_beta", 0.05)):
                niches, niching_matrix = niching.niche_finding(c_train_pred, y_train_one_hot, 'corr', threshold=beta)
                np_score = niching.niche_impurity(c_test_pred, y_test_one_hot, classifier, niches)
                current_niche_impurity_beta_scores.append(np_score['auc_impurity'])
                current_niche_size_beta_scores.append(niches.sum(axis=0).mean())
                
                # And update the area under the curve
                if 'niche_impurity' in prev_value_map:
                    current_niche_impurity_auc += (
                        prev_value_map['niche_impurity'] + np_score['auc_impurity']
                    ) * (delta_beta / 2)
                prev_value_map['niche_impurity'] = np_score['auc_impurity']
            
            print("\t\t\tReal Task Niche impurity by betas:", current_niche_impurity_beta_scores)
            print("\t\t\tReal Task Niche impurity AUC:", current_niche_impurity_auc)
            print("\t\t\tReal Task Niche average size:", current_niche_size_beta_scores)
            niche_impurity_aucs.append(current_niche_impurity_auc)
            niche_impurity_by_betas.append(current_niche_impurity_beta_scores)
            niche_size_by_betas.append(current_niche_size_beta_scores)
            
            
            # CONCEPT TASK NICHING SCORES
            classifier = MLPClassifier((20, 20), random_state=1, max_iter=1000).fit(c_train_pred, c_train)
            current_niche_impurity_beta_scores = []
            current_niche_size_beta_scores = []
            # And estimate the area under the curve using the trapezoid method
            current_niche_impurity_auc = 0
            prev_value_map = {}
            delta_beta = experiment_config.get("delta_beta", 0.05)
            for beta in np.arange(0.0, 1.0, experiment_config.get("delta_beta", 0.05)):
                niches, niching_matrix = niching.niche_finding(c_train_pred, c_train, 'corr', threshold=beta)
                np_score = niching.niche_impurity(c_test_pred, c_test, classifier, niches)
                current_niche_impurity_beta_scores.append(np_score['auc_impurity'])
                current_niche_size_beta_scores.append(niches.sum(axis=0).mean())

                # And update the area under the curve
                if 'niche_impurity' in prev_value_map:
                    current_niche_impurity_auc += (
                        prev_value_map['niche_impurity'] + np_score['auc_impurity']
                    ) * (delta_beta / 2)
                prev_value_map['niche_impurity'] = np_score['auc_impurity']

            print("\t\t\tReal Concept Niche impurity by betas:", current_niche_impurity_beta_scores)
            print("\t\t\tReal Concept Niche impurity AUC:", current_niche_impurity_auc)
            print("\t\t\tReal Concept Niche average size:", current_niche_size_beta_scores)
            concept_niche_impurity_aucs.append(current_niche_impurity_auc)
            concept_niche_impurity_by_betas.append(current_niche_impurity_beta_scores)
            concept_niche_size_by_betas.append(current_niche_size_beta_scores)
            
            
            print("\t\tComputing real intervention accuracies...")
            current_intervention_task_acc = []
            current_intervention_task_auc = []
            if experiment_config.get('random_intervention_order', False):
                intervention_order = np.random.permutation(experiment_config['num_concepts'])
            else:
                intervention_order = range(experiment_config['num_concepts'])
            for i in range(len(intervention_order) + 1):
                corrected_concepts = intervention_order[:i]
                intervention_result = cbm_intervention(
                    cbm_model=cbm_model,
                    x_train=x_train,
                    x_test=x_test,
                    y_test=y_test,
                    c_test=c_test[:, :experiment_config["num_concepts"]],
                    intervention_concepts=corrected_concepts,
                )
                current_intervention_task_acc.append(intervention_result['accuracy'])
                current_intervention_task_auc.append(intervention_result['auc'])
            
            print("\t\t\tReal Intervention AUCs:", current_intervention_task_auc)
            print("\t\t\tReal Intervention Accuraciess:", current_intervention_task_acc)
            intervention_task_accuracies.append(current_intervention_task_acc)
            intervention_task_aucs.append(current_intervention_task_auc)

            print(f"\t\tComputing real purity score...")
            soft_acts = (
                np.concatenate(cbm_model.encoder(x_test), axis=-1)
                if experiment_config["latent_dims"] else c_test_pred
            )
            purity_score, purity_mat, oracle_mat = purity.norm_purity_score(
                c_soft=soft_acts,
                c_true=c_test,
                output_matrices=True,
            )
            purity_mats.append(purity_mat)
            oracle_mats.append(oracle_mat)
            purities.append(purity_score)
            print(f"\t\t\tDone {purity_score:.4f}")

            print("\t\tComputing real non-oracle purity score...")
        
            non_oracle_purities.append(purity.norm_purity_score(
                c_soft=soft_acts,
                c_true=c_test,
                oracle_matrix=construct_trivial_auc_mat(
                    experiment_config["data_concepts"]
                ),
                purity_matrix=purity_mat,
            ))
            print(f"\t\t\tDone {non_oracle_purities[-1]:.4f}")
            
            
            
            print(f"\t\tComputing purity score (corrupted data)...")
            corrupted_c_train_pred = cbm_model.encoder(corrupted_x_train).numpy()
            corrupted_c_test_pred = cbm_model.encoder(corrupted_x_test).numpy()

            enc = OneHotEncoder(sparse=False)
            corrupted_y_train_one_hot = enc.fit_transform(corrupted_y_train.reshape(-1, 1))
            corrupted_y_test_one_hot = enc.fit_transform(corrupted_y_test.reshape(-1, 1))

            classifier = MLPClassifier((20, 20), random_state=1, max_iter=1000).fit(corrupted_c_train_pred, corrupted_y_train_one_hot)
            current_niche_impurity_beta_scores = []
            current_niche_size_beta_scores = []
            # And estimate the area under the curve using the trapezoid method
            current_niche_impurity_auc = 0
            prev_value_map = {}
            delta_beta = experiment_config.get("delta_beta", 0.05)
            for beta in np.arange(0.0, 1.0, experiment_config.get("delta_beta", 0.05)):
                niches, niching_matrix = niching.niche_finding(corrupted_c_train_pred, corrupted_y_train_one_hot, 'corr', threshold=beta)
                np_score = niching.niche_impurity(corrupted_c_test_pred, corrupted_y_test_one_hot, classifier, niches)
                current_niche_impurity_beta_scores.append(np_score['auc_impurity'])
                current_niche_size_beta_scores.append(niches.sum(axis=0).mean())
                
                # And update the area under the curve
                if 'niche_impurity' in prev_value_map:
                    current_niche_impurity_auc += (
                        prev_value_map['niche_impurity'] + np_score['auc_impurity']
                    ) * (delta_beta / 2)
                prev_value_map['niche_impurity'] = np_score['auc_impurity']
            
            print("\t\t\tCorrupted Task Niche impurity by betas:", current_niche_impurity_beta_scores)
            print("\t\t\tCorrupted Task Niche impurity AUC:", current_niche_impurity_auc)
            print("\t\t\tCorrupted Task Niche average size:", current_niche_size_beta_scores)
            corrupted_niche_impurity_aucs.append(current_niche_impurity_auc)
            corrupted_niche_impurity_by_betas.append(current_niche_impurity_beta_scores)
            corrupted_niche_size_by_betas.append(current_niche_size_beta_scores)
            
            
            # CONCEPT TASK NICHING SCORES
            classifier = MLPClassifier((20, 20), random_state=1, max_iter=1000).fit(corrupted_c_train_pred, corrupted_c_train)
            current_niche_impurity_beta_scores = []
            current_niche_size_beta_scores = []
            # And estimate the area under the curve using the trapezoid method
            current_niche_impurity_auc = 0
            prev_value_map = {}
            delta_beta = experiment_config.get("delta_beta", 0.05)
            for beta in np.arange(0.0, 1.0, experiment_config.get("delta_beta", 0.05)):
                niches, niching_matrix = niching.niche_finding(corrupted_c_train_pred, corrupted_c_train, 'corr', threshold=beta)
                np_score = niching.niche_impurity(corrupted_c_test_pred, corrupted_c_test, classifier, niches)
                current_niche_impurity_beta_scores.append(np_score['auc_impurity'])
                current_niche_size_beta_scores.append(niches.sum(axis=0).mean())

                # And update the area under the curve
                if 'niche_impurity' in prev_value_map:
                    current_niche_impurity_auc += (
                        prev_value_map['niche_impurity'] + np_score['auc_impurity']
                    ) * (delta_beta / 2)
                prev_value_map['niche_impurity'] = np_score['auc_impurity']

            print("\t\t\tCorrupted Concept Niche impurity by betas:", current_niche_impurity_beta_scores)
            print("\t\t\tCorrupted Concept Niche impurity AUC:", current_niche_impurity_auc)
            print("\t\t\tCorrupted Concept Niche average size:", current_niche_size_beta_scores)
            corrupted_concept_niche_impurity_aucs.append(current_niche_impurity_auc)
            corrupted_concept_niche_impurity_by_betas.append(current_niche_impurity_beta_scores)
            corrupted_concept_niche_size_by_betas.append(current_niche_size_beta_scores)
            
            
            print("\t\tComputing corrupted intervention accuracies...")
            current_intervention_task_acc = []
            current_intervention_task_auc = []
            if experiment_config.get('random_intervention_order', False):
                intervention_order = np.random.permutation(experiment_config['num_concepts'])
            else:
                intervention_order = range(experiment_config['num_concepts'])
            for i in range(len(intervention_order) + 1):
                corrected_concepts = intervention_order[:i]
                corrupted_intervention_result = cbm_intervention(
                    cbm_model=cbm_model,
                    x_train=corrupted_x_train,
                    x_test=corrupted_x_test,
                    y_test=corrupted_y_test,
                    c_test=corrupted_c_test[:, :experiment_config["num_concepts"]],
                    intervention_concepts=corrected_concepts,
                )
                current_intervention_task_acc.append(corrupted_intervention_result['accuracy'])
                current_intervention_task_auc.append(corrupted_intervention_result['auc'])
            
            print("\t\t\tCorrupted Intervention AUCs:", current_intervention_task_auc)
            print("\t\t\tCorrupted Intervention Accuraciess:", current_intervention_task_acc)
            corrupted_intervention_task_accuracies.append(current_intervention_task_acc)
            corrupted_intervention_task_aucs.append(current_intervention_task_auc)

            print(f"\t\tComputing corrupted purity score...")
            corrupted_soft_acts = (
                np.concatenate(cbm_model.encoder(corrupted_x_test), axis=-1)
                if experiment_config["latent_dims"] else corrupted_c_test_pred
            )
            corrupted_purity_score, corrupted_purity_mat, oracle_mat = purity.norm_purity_score(
                c_soft=corrupted_soft_acts,
                c_true=corrupted_c_test,
                output_matrices=True,
                oracle_matrix=oracle_mat,
            )
            corrupted_purity_mats.append(corrupted_purity_mat)
            corrupted_purities.append(corrupted_purity_score)
            print(f"\t\t\tDone {corrupted_purity_score:.4f}")
 
            
            print("\t\tDone with trial", trial + 1)

        task_acc_mean, task_acc_std = np.mean(task_accs), np.std(task_accs)
        experiment_variables["task_accuracies"].append((task_acc_mean, task_acc_std))
        print(f"\tReal Test task accuracy: {task_acc_mean:.4f} ± {task_acc_std:.4f}")
        
        corrupted_task_acc_mean, corrupted_task_acc_std = np.mean(corrupted_task_accs), np.std(corrupted_task_accs)
        experiment_variables["corrupted_task_accuracies"].append((corrupted_task_acc_mean, corrupted_task_acc_std))
        print(f"\tCorrupted Test task accuracy: {corrupted_task_acc_mean:.4f} ± {corrupted_task_acc_std:.4f}")

        
        
        concept_acc_mean, concept_acc_std = np.mean(concept_accs), np.std(concept_accs)
        experiment_variables["concept_accuracies"].append((concept_acc_mean, concept_acc_std))
        print(f"\tTest concept accuracy: {concept_acc_mean:.4f} ± {concept_acc_std:.4f}")

        corrupted_concept_acc_mean, corrupted_concept_acc_std = np.mean(corrupted_concept_accs), np.std(corrupted_concept_accs)
        experiment_variables["corrupted_concept_accuracies"].append((corrupted_concept_acc_mean, corrupted_concept_acc_std))
        print(f"\tCorrupted Test concept accuracy: {corrupted_concept_acc_mean:.4f} ± {corrupted_concept_acc_std:.4f}")


        
        task_auc_mean, task_auc_std = np.mean(aucs), np.std(aucs)
        experiment_variables["task_aucs"].append((task_auc_mean, task_auc_std))
        print(f"\tReal Test task AUC: {task_auc_mean:.4f} ± {task_auc_std:.4f}")
        
        corrupted_task_auc_mean, corrupted_task_auc_std = np.mean(corrupted_aucs), np.std(corrupted_aucs)
        experiment_variables["corrupted_task_aucs"].append((corrupted_task_auc_mean, corrupted_task_auc_std))
        print(f"\tCorrupted Test task AUC: {corrupted_task_auc_mean:.4f} ± {corrupted_task_auc_std:.4f}")

        
        
        purity_mats = np.stack(purity_mats, axis=0)
        purity_mat_mean = np.mean(purity_mats, axis=0)
        purity_mat_std = np.std(purity_mats, axis=0)
        print("\tReal Purity matrix:")
        for i in range(purity_mat_mean.shape[0]):
            line = "\t\t"
            for j in range(purity_mat_mean.shape[1]):
                line += f'{purity_mat_mean[i, j]:.4f} ± {purity_mat_std[i, j]:.4f}    '
            print(line)

        experiment_variables["purity_matrices"].append((purity_mat_mean, purity_mat_std))
        
        corrupted_purity_mats = np.stack(corrupted_purity_mats, axis=0)
        corrupted_purity_mat_mean = np.mean(corrupted_purity_mats, axis=0)
        corrupted_purity_mat_std = np.std(corrupted_purity_mats, axis=0)
        print("\tCorrupted Purity matrix:")
        for i in range(corrupted_purity_mat_mean.shape[0]):
            line = "\t\t"
            for j in range(corrupted_purity_mat_mean.shape[1]):
                line += f'{corrupted_purity_mat_mean[i, j]:.4f} ± {corrupted_purity_mat_std[i, j]:.4f}    '
            print(line)

        experiment_variables["corrupted_purity_matrices"].append((corrupted_purity_mat_mean, corrupted_purity_mat_std))


        
        
        oracle_mats = np.stack(oracle_mats, axis=0)
        oracle_mat_mean = np.mean(oracle_mats, axis=0)
        oracle_mat_std = np.std(oracle_mats, axis=0)
        print("\tOracle matrix:")
        for i in range(oracle_mat_mean.shape[0]):
            line = "\t\t"
            for j in range(oracle_mat_mean.shape[1]):
                line += f'{oracle_mat_mean[i, j]:.4f} ± {oracle_mat_std[i, j]:.4f}    '
            print(line)

        experiment_variables["oracle_matrices"].append((oracle_mat_mean, oracle_mat_std))

        
        
        purity_mean, purity_std = np.mean(purities), np.std(purities)
        experiment_variables["purity_scores"].append((purity_mean, purity_std))
        print(f"\tReal Purity score: {purity_mean:.4f} ± {purity_std:.4f}")
        
        corrupted_purity_mean, corrupted_purity_std = np.mean(corrupted_purities), np.std(corrupted_purities)
        experiment_variables["corrupted_purity_scores"].append((corrupted_purity_mean, corrupted_purity_std))
        print(f"\tCorrupted Purity score: {corrupted_purity_mean:.4f} ± {corrupted_purity_std:.4f}")

        
        
        
        non_oracle_purity_mean, non_oracle_purity_std = np.mean(non_oracle_purities), np.std(non_oracle_purities)
        experiment_variables["non_oracle_purity_scores"].append((non_oracle_purity_mean, non_oracle_purity_std))
        print(f"\tNon-oracle purity score: {non_oracle_purity_mean:.4f} ± {non_oracle_purity_std:.4f}")
        
        
        
        # Time for niche impurity scores
        niche_impurity_hist = np.stack(niche_impurity_by_betas, axis=0)
        niche_impurity_mean = np.mean(niche_impurity_hist, axis=0)
        niche_impurity_std = np.std(niche_impurity_hist, axis=0)
        experiment_variables["niche_impurity_by_betas"].append((niche_impurity_mean, niche_impurity_std))
        line = "\tReal Task Niche Impurities by Beta: ["
        for j in range(niche_impurity_mean.shape[0]):
            line += f'{niche_impurity_mean[j]:.4f} ± {niche_impurity_std[j]:.4f},  '
        print(line + "]")
        
        corrupted_niche_impurity_hist = np.stack(corrupted_niche_impurity_by_betas, axis=0)
        corrupted_niche_impurity_mean = np.mean(corrupted_niche_impurity_hist, axis=0)
        corrupted_niche_impurity_std = np.std(corrupted_niche_impurity_hist, axis=0)
        experiment_variables["corrupted_niche_impurity_by_betas"].append((corrupted_niche_impurity_mean, corrupted_niche_impurity_std))
        line = "\tCorrupted Task Niche Impurities by Beta: ["
        for j in range(corrupted_niche_impurity_mean.shape[0]):
            line += f'{corrupted_niche_impurity_mean[j]:.4f} ± {corrupted_niche_impurity_std[j]:.4f},  '
        print(line + "]")
        
        concept_niche_impurity_hist = np.stack(concept_niche_impurity_by_betas, axis=0)
        concept_niche_impurity_mean = np.mean(concept_niche_impurity_hist, axis=0)
        concept_niche_impurity_std = np.std(concept_niche_impurity_hist, axis=0)
        experiment_variables["concept_niche_impurity_by_betas"].append((concept_niche_impurity_mean, concept_niche_impurity_std))
        line = "\tReal Concept Niche Impurities by Beta: ["
        for j in range(concept_niche_impurity_mean.shape[0]):
            line += f'{concept_niche_impurity_mean[j]:.4f} ± {concept_niche_impurity_std[j]:.4f},  '
        print(line + "]")
        
        corrupted_concept_niche_impurity_hist = np.stack(corrupted_concept_niche_impurity_by_betas, axis=0)
        corrupted_concept_niche_impurity_mean = np.mean(corrupted_concept_niche_impurity_hist, axis=0)
        corrupted_concept_niche_impurity_std = np.std(corrupted_concept_niche_impurity_hist, axis=0)
        experiment_variables["corrupted_concept_niche_impurity_by_betas"].append((corrupted_concept_niche_impurity_mean, corrupted_concept_niche_impurity_std))
        line = "\tCorrupted Concept Niche Impurities by Beta: ["
        for j in range(corrupted_concept_niche_impurity_mean.shape[0]):
            line += f'{corrupted_concept_niche_impurity_mean[j]:.4f} ± {corrupted_concept_niche_impurity_std[j]:.4f},  '
        print(line + "]")
        
        
        
        
        niche_size_hist = np.stack(niche_size_by_betas, axis=0)
        niche_size_mean = np.mean(niche_size_hist, axis=0)
        niche_size_std = np.std(niche_size_hist, axis=0)
        experiment_variables["niche_size_by_betas"].append((niche_size_mean, niche_size_std))
        line = "\tReal Task Niche Size by Beta: ["
        for j in range(niche_size_mean.shape[0]):
            line += f'{niche_size_mean[j]:.4f} ± {niche_size_std[j]:.4f},  '
        print(line + "]")
        
        corrupted_niche_size_hist = np.stack(corrupted_niche_size_by_betas, axis=0)
        corrupted_niche_size_mean = np.mean(corrupted_niche_size_hist, axis=0)
        corrupted_niche_size_std = np.std(corrupted_niche_size_hist, axis=0)
        experiment_variables["corrupted_niche_size_by_betas"].append((corrupted_niche_size_mean, corrupted_niche_size_std))
        line = "\tCorrupted Task Niche Size by Beta: ["
        for j in range(corrupted_niche_size_mean.shape[0]):
            line += f'{corrupted_niche_size_mean[j]:.4f} ± {corrupted_niche_size_std[j]:.4f},  '
        print(line + "]")
        
        concept_niche_size_hist = np.stack(concept_niche_size_by_betas, axis=0)
        concept_niche_size_mean = np.mean(concept_niche_size_hist, axis=0)
        concept_niche_size_std = np.std(concept_niche_size_hist, axis=0)
        experiment_variables["niche_size_by_betas"].append((concept_niche_size_mean, concept_niche_size_std))
        line = "\tReal Concept Niche Size by Beta: ["
        for j in range(concept_niche_size_mean.shape[0]):
            line += f'{concept_niche_size_mean[j]:.4f} ± {concept_niche_size_std[j]:.4f},  '
        print(line + "]")
        
        corrupted_concept_niche_size_hist = np.stack(corrupted_concept_niche_size_by_betas, axis=0)
        corrupted_concept_niche_size_mean = np.mean(corrupted_concept_niche_size_hist, axis=0)
        corrupted_concept_niche_size_std = np.std(corrupted_concept_niche_size_hist, axis=0)
        experiment_variables["corrupted_concept_niche_size_by_betas"].append((corrupted_concept_niche_size_mean, corrupted_concept_niche_size_std))
        line = "\tCorrupted Concept Niche Size by Beta: ["
        for j in range(corrupted_concept_niche_size_mean.shape[0]):
            line += f'{corrupted_concept_niche_size_mean[j]:.4f} ± {corrupted_concept_niche_size_std[j]:.4f},  '
        print(line + "]")
        
        
        
        
        niche_impurity_auc_mean, niche_impurity_auc_std = np.mean(niche_impurity_aucs), np.std(niche_impurity_aucs)
        experiment_variables["niche_impurity_aucs"].append((niche_impurity_auc_mean, niche_impurity_auc_std))
        print(f"\tReal Task Niche impurity AUC: {niche_impurity_auc_mean:.4f} ± {niche_impurity_auc_std:.4f}")
        
        corrupted_niche_impurity_auc_mean, corrupted_niche_impurity_auc_std = np.mean(corrupted_niche_impurity_aucs), np.std(corrupted_niche_impurity_aucs)
        experiment_variables["corrupted_niche_impurity_aucs"].append((corrupted_niche_impurity_auc_mean, corrupted_niche_impurity_auc_std))
        print(f"\tCorrupted Task Niche impurity AUC: {corrupted_niche_impurity_auc_mean:.4f} ± {corrupted_niche_impurity_auc_std:.4f}")
        
        concept_niche_impurity_auc_mean, concept_niche_impurity_auc_std = np.mean(concept_niche_impurity_aucs), np.std(concept_niche_impurity_aucs)
        experiment_variables["concept_niche_impurity_aucs"].append((concept_niche_impurity_auc_mean, concept_niche_impurity_auc_std))
        print(f"\tReal Concept Niche impurity AUC: {concept_niche_impurity_auc_mean:.4f} ± {concept_niche_impurity_auc_std:.4f}")
        
        corrupted_concept_niche_impurity_auc_mean, corrupted_concept_niche_impurity_auc_std = np.mean(corrupted_concept_niche_impurity_aucs), np.std(corrupted_concept_niche_impurity_aucs)
        experiment_variables["corrupted_concept_niche_impurity_aucs"].append((corrupted_concept_niche_impurity_auc_mean, corrupted_concept_niche_impurity_auc_std))
        print(f"\tCorrupted Concept Niche impurity AUC: {corrupted_concept_niche_impurity_auc_mean:.4f} ± {corrupted_concept_niche_impurity_auc_std:.4f}")
        
        
        
        # Finally, we end with intervention accuracies
        interventaion_auc_hist = np.stack(intervention_task_aucs, axis=0)
        interventaion_auc_mean = np.mean(interventaion_auc_hist, axis=0)
        interventaion_auc_std = np.std(interventaion_auc_hist, axis=0)
        experiment_variables["intervention_task_aucs"].append((interventaion_auc_mean, interventaion_auc_std))
        line = "\tReal Intervention AUCs: ["
        for j in range(interventaion_auc_mean.shape[0]):
            line += f'{interventaion_auc_mean[j]:.4f} ± {interventaion_auc_std[j]:.4f},  '
        print(line + "]")
        
        # Finally, we end with intervention accuracies
        corrupted_interventaion_auc_hist = np.stack(corrupted_intervention_task_aucs, axis=0)
        corrupted_interventaion_auc_mean = np.mean(corrupted_interventaion_auc_hist, axis=0)
        corrupted_interventaion_auc_std = np.std(corrupted_interventaion_auc_hist, axis=0)
        experiment_variables["corrupted_intervention_task_aucs"].append((corrupted_interventaion_auc_mean, corrupted_interventaion_auc_std))
        line = "\tCorrupted Intervention AUCs: ["
        for j in range(corrupted_interventaion_auc_mean.shape[0]):
            line += f'{corrupted_interventaion_auc_mean[j]:.4f} ± {corrupted_interventaion_auc_std[j]:.4f},  '
        print(line + "]")
        
        
        
        
        interventaion_acc_hist = np.stack(intervention_task_accuracies, axis=0)
        interventaion_acc_mean = np.mean(interventaion_acc_hist, axis=0)
        interventaion_acc_std = np.std(interventaion_acc_hist, axis=0)
        experiment_variables["intervention_task_accuracies"].append((interventaion_acc_mean, interventaion_acc_std))
        line = "\tIntervention Accuracies: ["
        for j in range(interventaion_acc_mean.shape[0]):
            line += f'{interventaion_acc_mean[j]:.4f} ± {interventaion_acc_std[j]:.4f},  '
        print(line + "]")
        
        corrupted_interventaion_acc_hist = np.stack(corrupted_intervention_task_accuracies, axis=0)
        corrupted_interventaion_acc_mean = np.mean(corrupted_interventaion_acc_hist, axis=0)
        corrupted_interventaion_acc_std = np.std(corrupted_interventaion_acc_hist, axis=0)
        experiment_variables["corrupted_intervention_task_accuracies"].append((corrupted_interventaion_acc_mean, corrupted_interventaion_acc_std))
        line = "\tCorrupted Intervention Accuracies: ["
        for j in range(corrupted_interventaion_acc_mean.shape[0]):
            line += f'{corrupted_interventaion_acc_mean[j]:.4f} ± {corrupted_interventaion_acc_std[j]:.4f},  '
        print(line + "]")
        
        
        
        
        # Increase the experiment counter
        experiment_variables["current_experiment_idx"] += 1

        # And serialize the results
        joblib.dump(
            experiment_variables,
            os.path.join(experiment_config["results_dir"], 'results.joblib'),
        )
    return experiment_variables

## Run time!

In [ ]:
reload(purity)
reload(CBM)

############################################################################
## Experiment config
############################################################################

color_random_from_probs_experiment_config = dict(
    trials=5,
    batch_size=64 if USE_DSPRITES else 32,
    max_epochs=100,
    pre_train_epochs=0,
    warmup_epochs=0,
    learning_rate=1e-3,
    encoder_filter_groups=[
        [(8, (7, 7))],
        [(16, (5, 5))],
        [(32, (3, 3))],
        [(64, (3, 3))]
    ],
    encoder_units=[64, 64],
    decoder_units=[64, 64],
    alpha_values=[0.1, 10],

    latent_decoder_units=[64, 64],
    predictor_max_epochs=100,
    concept_predictor_max_epochs=100,
    
    drop_prob=0.5,
    max_pool_window=(2,2),
    pax_pool_stride=2,
    num_outputs=(
        len(set(color_random_corrupted_data_train[1])) if len(set(color_random_corrupted_data_train[1])) > 2
        else 1
    ),
    concept_cardinality=[1 for _ in range(color_random_corrupted_data_train[2].shape[-1])],
    patience=float("inf"),
    min_delta=1e-5,
    results_dir=os.path.join(RESULTS_DIR, "cbm/color_random_base_from_probs"),
    input_shape=color_random_corrupted_data_train[0].shape[1:],
    num_concepts=color_random_corrupted_data_train[2].shape[-1],
    latent_dims=0,
    holdout_fraction=0.1,
    verbosity=0,
    early_stop_metric="val_concept_accuracy",
    early_stop_mode="max",
    encoder_output_logits=False,
    delta_beta=0.05,
)

############################################################################
## Experiment run
############################################################################

color_random_from_probs_results = cbm_mixed_capacity_intervention_experiment_loop(
    train_data=data_train,
    corrupted_train_data=color_random_corrupted_data_train,
    test_data=data_test,
    corrupted_test_data=color_random_corrupted_data_test,
    experiment_config=color_random_from_probs_experiment_config,
    load_from_cache=True,
)
print("task_accuracies:", color_random_from_probs_results["task_accuracies"])
print("corrupted_task_accuracies:", color_random_from_probs_results["corrupted_task_accuracies"])
print("concept_accuracies:", color_random_from_probs_results["concept_accuracies"])
print("corrupted_concept_accuracies:", color_random_from_probs_results["corrupted_concept_accuracies"])
print("task_aucs:", color_random_from_probs_results["task_aucs"])
print("purity_scores:", color_random_from_probs_results["purity_scores"])
print("corrupted_purity_scores:", color_random_from_probs_results["corrupted_purity_scores"])
print("concept_niche_impurity_aucs:", color_random_from_probs_results["concept_niche_impurity_aucs"])
print("corrupted_concept_niche_impurity_aucs:", color_random_from_probs_results["corrupted_concept_niche_impurity_aucs"])


In [ ]:
reload(purity)
reload(CBM)

############################################################################
## Experiment config
############################################################################

base_color_experiment_config = dict(
    trials=5,
    batch_size=64 if USE_DSPRITES else 32,
    max_epochs=100,
    pre_train_epochs=0,
    warmup_epochs=0,
    learning_rate=1e-3,
    encoder_filter_groups=[
        [(8, (7, 7))],
        [(16, (5, 5))],
        [(32, (3, 3))],
        [(64, (3, 3))]
    ],
    encoder_units=[64, 64],
    decoder_units=[64, 64],
    alpha_values=[0.0, 0.01, 0.1, 1], #, 10],

    latent_decoder_units=[64, 64],
    predictor_max_epochs=100,
    concept_predictor_max_epochs=100,
    
    drop_prob=0.5,
    max_pool_window=(2,2),
    pax_pool_stride=2,
    num_outputs=(
        len(set(data_train[1])) if len(set(data_train[1])) > 2
        else 1
    ),
    concept_cardinality=[1 for _ in range(data_train[2].shape[-1])],
    patience=float("inf"),
    min_delta=1e-5,
    results_dir=os.path.join(RESULTS_DIR, "cbm/base_color"),
    input_shape=data_train[0].shape[1:],
    num_concepts=data_train[2].shape[-1],
    latent_dims=0,
    holdout_fraction=0.1,
    verbosity=0,
    early_stop_metric="val_concept_accuracy",
    early_stop_mode="max",
    encoder_output_logits=False,
    delta_beta=0.05,
)

############################################################################
## Experiment run
############################################################################

base_color_results = cbm_mixed_capacity_intervention_experiment_loop(
    train_data=data_train,
    corrupted_train_data=color_corrupted_data_train,
    test_data=data_test,
    corrupted_test_data=color_corrupted_data_test,
    experiment_config=base_color_experiment_config,
    load_from_cache=True,
)
print("task_accuracies:", base_color_results["task_accuracies"])
print("corrupted_task_accuracies:", base_color_results["corrupted_task_accuracies"])
print("concept_accuracies:", base_color_results["concept_accuracies"])
print("corrupted_concept_accuracies:", base_color_results["corrupted_concept_accuracies"])
print("task_aucs:", base_color_results["task_aucs"])
print("purity_scores:", base_color_results["purity_scores"])
print("corrupted_purity_scores:", base_color_results["purity_scores"])
print("corrupted_concept_niche_impurity_aucs:", base_color_results["corrupted_concept_niche_impurity_aucs"])
